In [39]:
# get_lof.py
# Usage: get_lof.py FE_plink.chr21.recode.vep.vcf.gz chr21

import sys
from cyvcf2 import VCF
import time
import logging
logging.basicConfig(format='%(message)s')

logging.warning('Starting. Args={}'.format(sys.argv))



05/09/2021 01:08:20 PM Starting. Args=['/hps/software/users/birney/shimin/miniconda3/envs/cyvcf/lib/python3.9/site-packages/ipykernel_launcher.py', '/nfs/research/birney/users/shimin/ukbb/wes/200k/pvcf/4_vep/ukb23156_c10_b0_v1.vep.qc.norm.vcf.bgz', 'test2']


In [33]:
sys.argv[1] = '/nfs/research/birney/users/shimin/ukbb/wes/200k/pvcf/4_vep/ukb23156_c10_b0_v1.vep.qc.norm.vcf.bgz'
sys.argv[2] = 'test2'

vcf_reader = VCF(sys.argv[1], gts012=True, threads=4)
n_record = 0
n_mask1 = 0
n_mask2 = 0

In [36]:
info_format = vcf_reader.get_header_type('CSQ')['Description'].split('Format: ')[1].split('|')
info_format[-1] = 'LoF_info'
# print(info_format)
max_af = info_format.index('MAX_AF')
lof = info_format.index('LoF')
canonical = info_format.index('CANONICAL')
lrt_pred = info_format.index('LRT_pred')
consequence = info_format.index('Consequence')
impact = info_format.index('IMPACT')

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'AA_AF', 'EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'TRANSCRIPTION_FACTORS', 'Ensembl_transcriptid', 'LRT_pred', 'MutationTaster_pred', 'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_pred', 'SIFT_

In [34]:
def apply_mask(csq_records):
    global max_af, lof, canonical, impact, consequence
    for csq in csq_records:
        csq_info = csq.split('|')
        maf = 0 if csq_info[max_af] == '' else float(csq_info[max_af])
        # Rare, canonical
        if csq_info[canonical] == 'YES' and maf < 0.01:
            if csq_info[lof] == 'HC':
                return('m1', csq_info)
            elif csq_info[consequence] == 'missense_variant' and (csq_info[impact] in ('HIGH', 'MODERATE')):
                return('m2', csq_info)
    return(None, csq_info)


# Generate genotype matrix
fgt_m1 = open(sys.argv[2] + '.mask1.genotype.tsv', 'w')
fgt_m2 = open(sys.argv[2] + '.mask2.genotype.tsv', 'w')
# Write header of sample names.
fgt_m1.write("\t".join(['CHROM', 'POS', 'ID', 'REF', 'ALT'] + info_format + vcf_reader.samples) + '\n')
fgt_m2.write("\t".join(['CHROM', 'POS', 'ID', 'REF', 'ALT'] + info_format + vcf_reader.samples) + '\n')

    
# Input VCF with VEP annotated variants
t = time.process_time()
for record in vcf_reader:
    n_record += 1
    if n_record % 1000 == 0:
        delta_t = time.process_time() - t
        logging.warning('Processing {} records, {} secs passed.'.format(n_record, delta_t))
    if record.aaf > 0.01:
        # Not rare in the current cohort, skip
        continue
    mask, info = apply_mask(record.INFO['CSQ'].split(','))
    if mask == 'm1':
        n_mask1 += 1
        # The type of genotype. hom_ref = 0 het = 1 hom_alt = 2 uncalled = None
        fgt_m1.write("\t".join([record.CHROM, str(record.POS), record.ID, record.REF, record.ALT[0]] + info + record.gt_types.astype(str).tolist()) + '\n')
    elif mask == 'm2':
        n_mask2 += 1
        fgt_m2.write("\t".join([record.CHROM, str(record.POS), record.ID, record.REF, record.ALT[0]] + info + record.gt_types.astype(str).tolist()) + '\n')

fgt_m1.close()
fgt_m2.close()

logging.warning(f'{sys.argv[2]},stats,n_mask1,n_mask2,n_record')
logging.warning(f'{sys.argv[2]},stats,{n_mask1},{n_mask2},{n_record}')

05/09/2021 12:54:54 PM Processing 1000 records, 99.96923702 secs passed.
05/09/2021 12:56:30 PM Processing 2000 records, 201.357912156 secs passed.
05/09/2021 12:58:12 PM Processing 3000 records, 308.562281291 secs passed.
05/09/2021 12:59:50 PM Processing 4000 records, 411.845951862 secs passed.
05/09/2021 01:01:23 PM Processing 5000 records, 510.42041390799994 secs passed.
05/09/2021 01:03:05 PM Processing 6000 records, 617.5556414680001 secs passed.
05/09/2021 01:04:52 PM Processing 7000 records, 729.632634853 secs passed.
05/09/2021 01:05:01 PM Stats,n_mask1,n_mask2,n_record
05/09/2021 01:05:01 PM Stats,210,3329,7081
